In [2]:
# ! pip install -U finance-datareader

In [1]:
import pandas as pd
import os
import FinanceDataReader as fdr
from tqdm import tqdm

# 과제 1

- 종목코드

In [3]:
import pandas as pd
df_stock = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download', header=0)[0] #데이터 불러오기
df_stock['종목코드'] = df_stock['종목코드'].apply(lambda x : str(x).zfill(6))
lst_stock = df_stock.values.tolist()
# print(lst_stock [:10]) 

In [4]:
OF=open('assignment1-3.txt','w') 
lst_code=[]  
for row in lst_stock: 
    date = row[4]  
    if date <= '2018-01-01': 
        lst_code.append(row[1])  
        OF.write(f'{row[1]}\n') 
OF.close()
print(lst_code[:10]) 

['000210', '004840', '155660', '078930', '001250', '039570', '011200', '092220', '016380', '001390']


--------

- 종목코드 + 회사명 

In [5]:
# 종목코드만 리스트 저장 + 파일 쓰기
lst_code = []
OF = open("assignment1-4.txt", 'w', encoding='UTF-8')
for row in lst_stock: 
    name, code, date = row[0], row[1], row[4] 
    if date <= '2018-01-01': 
        lst_code.append(code)
        OF.write(f'{code}\t{name}\n')

------

In [19]:
# dictionary 저장 후 파일 쓰기
dic_code2name = {}
for row in lst_stock:
    name, code, date = row[0], row[1], row[4]
    if date <= '2018-01-01': 
        dic_code2name[code] = name 

In [20]:
OF = open('assignment1-5.txt', 'w', encoding='UTF-8')
for cd in dic_code2name: 
    OF.write(f'{cd}\t{dic_code2name[cd]}\n')

# 과제 2 

In [6]:
import time, os
from multiprocessing import Pool

In [7]:
dict_result = {}
OF = open('assignment2-2.txt','w')


for cd in tqdm(lst_code): 
    start_date = '20180102'
    end_date = '20201231'
    stock = fdr.DataReader(cd, start = start_date, end = end_date)
    stock.reset_index(inplace=True)
    
    # dataframe -> list
    lst_stock = stock.values.tolist()
    
    
    for stock_val in lst_stock: 
        if stock_val[4] * stock_val[5] >= 10000000000: 
            if cd not in dict_result.keys():
                dict_result[cd] = []
                dict_result[cd].append(stock_val[0].date().strftime("%Y%m%d"))
            else: 
                dict_result[cd].append(stock_val[0].date().strftime("%Y%m%d"))
                
            OF.write(f'{cd}\t{stock_val[0].date().strftime("%Y%m%d")}\n')
#             OF.write('\t'.join(map(str,stock_val))+'\n') # 변수 많을 때 join 사용 
            OF.flush()  # 중간 저장
    
OF.close()

100%|██████████████████████████████████████████████████████████████████████████████| 1999/1999 [11:39<00:00,  2.86it/s]


In [8]:
dict_result

{'000210': ['20180104',
  '20180105',
  '20180108',
  '20180109',
  '20180110',
  '20180111',
  '20180112',
  '20180116',
  '20180119',
  '20180125',
  '20180126',
  '20180129',
  '20180130',
  '20180131',
  '20180201',
  '20180202',
  '20180205',
  '20180206',
  '20180207',
  '20180208',
  '20180209',
  '20180212',
  '20180213',
  '20180222',
  '20180226',
  '20180302',
  '20180307',
  '20180308',
  '20180309',
  '20180312',
  '20180313',
  '20180327',
  '20180328',
  '20180329',
  '20180409',
  '20180410',
  '20180411',
  '20180412',
  '20180413',
  '20180418',
  '20180419',
  '20180420',
  '20180423',
  '20180424',
  '20180425',
  '20180426',
  '20180427',
  '20180430',
  '20180502',
  '20180503',
  '20180504',
  '20180508',
  '20180509',
  '20180510',
  '20180511',
  '20180514',
  '20180515',
  '20180516',
  '20180517',
  '20180518',
  '20180521',
  '20180523',
  '20180525',
  '20180528',
  '20180529',
  '20180530',
  '20180531',
  '20180601',
  '20180604',
  '20180605',
  '2018060

# 과제 3

In [9]:
# 종목코드 별 stock 뽑기
OF = open('assignment3-3.txt', 'w')
for code in tqdm(dict_result): 
    # code의 stock 
    start_date = '20171201' 
    end_date = '20210130' 
    stock = fdr.DataReader(code, start = start_date, end = end_date)
    stock.reset_index(inplace=True)
    lst_stock = stock.values.tolist()  
    
    
    for d0_date in dict_result[code]:
        for idx, stock_val in enumerate(lst_stock):
            
            if stock_val[0].date().strftime("%Y%m%d") == d0_date: 
                D9_idx = idx-9
                next_idx = idx+1
            
                # 10일간의 데이터 (info)
                info_list = []
                for day_info in lst_stock[D9_idx: next_idx]:
                    info_list += day_info[1:5] + [(day_info[4] * day_info[5])]
                    info = ','.join(map(str, info_list))

                # D+1 종가 2% 상승 여부 (up) 
                up = int(lst_stock[next_idx][-1] > 0.02)
                
                # 저장 
                OF.write(f'{code}\t{d0_date}\t{info}\t{up}\n')
                         
OF.close()               

100%|██████████████████████████████████████████████████████████████████████████████| 1790/1790 [21:52<00:00,  1.36it/s]
